In [1]:
from pathlib import Path
import sys
sys.path.append(str(Path().resolve().parent))
import os

In [2]:
import csv
from typing import Tuple

import numpy as np
from qulacs import Observable
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

from scikit_quri.circuit.pre_defined import create_dqn_cl, create_dqn_cl_no_cz
from scikit_quri.circuit import LearningCircuit
from scikit_quri.qnn.classifier import QNNClassifier
from quri_parts.core.estimator.gradient import (
    create_numerical_gradient_estimator,
    create_parameter_shift_gradient_estimator)
from quri_parts.qulacs.estimator import (
    create_qulacs_vector_concurrent_estimator,
    create_qulacs_vector_concurrent_parametric_estimator,
)
from quri_parts.algo.optimizer import Adam
from quri_parts.core.operator import Operator, pauli_label
# from scikit_quri.qnn.solver import Adam

In [3]:
# Use wine dataset retrieved from: https://archive-beta.ics.uci.edu/ml/datasets/wine
def load_dataset(
    file_path: str, ignore_kind: int, test_ratio: float
) -> Tuple[np.array, np.array, np.array, np.array]:
    """Load dataset from specified path.

    Args:
        file_path: File path from which data is loaded.
        ignore_kind: The dataset expected to have 3 classes and we need 2 classes to test. So specify here which class to ignore in loading.
    """
    x = []
    y = []
    with open(file_path) as f:
        reader = csv.reader(f)
        for row in reader:
            kind = int(row[0])
            if kind == ignore_kind:
                continue
            y.append(kind)
            x.append([float(feature) for feature in row[1:]])

    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=test_ratio, shuffle=True
    )

    return x_train, x_test, y_train, y_test

In [4]:
def create_classifier(n_features: int, circuit:LearningCircuit, locality:int):
    adam = Adam()
    estimator = create_qulacs_vector_concurrent_estimator()
    gradient_estimator = create_numerical_gradient_estimator(create_qulacs_vector_concurrent_parametric_estimator(),delta=1e-3)
    ops = []
    for i in range(n_features):
        if i < locality:
            op = Operator({pauli_label(f"Z {i}"):1.0})
        # else:
        #     op = Operator({pauli_label(f"I {i}"):1.0}) 
        ops.append(op)

    classifier = QNNClassifier(circuit,2,estimator,gradient_estimator,optimizer=adam,operator=ops)
    return classifier


In [ ]:
x_train, x_test, y_train, y_test = load_dataset("../datasets/wine.data", 3, 0.5)
for i in range(len(y_train)):
    y_train[i] -= 1
for i in range(len(y_test)):
    y_test[i] -= 1
from icecream import ic

n_features = 13
locality = 2
maxiter = 5

print("CL")
circuit = create_dqn_cl(n_features, 5, locality)
classifier = create_classifier(n_features,circuit,locality)
result_cl = []
loop_size = 80
for i in range(loop_size):
    classifier.fit(np.array(x_train), np.array(y_train), maxiter)
    ic(len(np.array(x_train)))
    ic(len(np.array(y_train)))
    y_pred = classifier.predict(np.array(x_test))
    score = f1_score(y_test, y_pred, average="weighted")
    result_cl.append(score)
    print(f"iteration:{(i+1)*maxiter} score:{score}")

print("no CL")
circuit = create_dqn_cl_no_cz(n_features, 5)
classifier = create_classifier(n_features,circuit, locality)
result_no_cl = []
for i in range(loop_size):
    classifier.fit(np.array(x_train),np.array(y_train), maxiter)
    y_pred = classifier.predict(np.array(x_test))
    score = f1_score(y_test, y_pred, average="weighted")
    result_no_cl.append(score)
    print(f"iteration:{(i+1)*maxiter} score:{score}")

CL
init_params=array([3.56758334, 3.9301883 , 0.94890649, 0.30665025, 0.22143148,
       1.78699687, 3.68908746, 6.14882607, 0.58043035, 2.03461304,
       5.10317893, 3.56739289, 6.23404951, 1.67454045, 5.73624997,
       1.52885996, 3.80560108, 3.08897536, 5.08381035, 0.97799882,
       5.54829161, 5.4352653 , 0.62137329, 5.58994788, 6.02859694,
       1.02418806, 4.92409706, 3.21032465, 4.29586258, 4.59057592,
       4.56698698, 1.39074706, 3.11682219, 5.62144216, 0.12895725,
       4.46611901, 1.98728065, 4.25849649, 4.43126296, 2.45840479,
       0.00940646, 4.34228735, 3.47617579, 1.18789041, 5.59870164,
       3.96991619, 0.59688731, 4.18567582, 3.23608334, 4.60881889,
       5.55322806, 4.95948869, 4.61024587, 5.33572936, 0.70422948,
       5.66427358, 2.0663358 , 3.64419473, 3.79105809, 4.70812692,
       1.63359728, 1.87705153, 2.22384299, 3.52935338, 2.07050661,
       3.54595297, 2.53438864, 3.28465887, 1.2032839 , 3.62633987,
       6.26843265, 1.1861081 , 5.22975496, 0.44

In [ ]:
import matplotlib.pyplot as plt

plt.plot(np.arange(0, loop_size*maxiter, step=maxiter), np.array(result_cl).flatten(), label="CL")
plt.plot(np.arange(0, loop_size*maxiter, step=maxiter), np.array(result_no_cl).flatten(), label="no CL")
plt.xticks(np.arange(0, loop_size*maxiter, step=25))
plt.xlabel("iteration")
plt.ylabel("score")
plt.legend()
plt.show()